In [6]:
import pandas as pd
import numpy as np
import os

In [7]:
#url = "https://raw.githubusercontent.com/adam-datasci/RiskAnalytics/main/Data/american_bankruptcy.csv?"
url = "Data/american_bankruptcy_kaggle.csv"
df = pd.read_csv(url)

df = df.rename(columns={'X1':'ca','X2':'cogs','X3':'d_and_a','X4':'ebitda','X5':'inventory','X6':'ni','X7':'ar','X8':'mv','X9':'ns','X10':'ta','X11':'tltd','X12':'ebit','X13':'gp','X14':'cl','X15':'re','X16':'tr','X17':'tl','X18':'toe'})

In [8]:
df['te'] = df['ta'] - df['tl']
df['wc'] = df['ca'] - df['cl']
df['wc_to_ta'] = df['wc'] / df['ta']
df['re_to_ta'] = df['re'] / df['ta']
df['ebit_to_ta'] = df['ebit'] / df['ta']
df['mv_to_tl'] = df['mv'] / df['tl']
df['tr_to_ta'] = df['tr'] / df['ta']
df['tltd_to_te'] = df['tltd'] / df['te']
df['np'] = df['ni']/df['tr']
df['ebitda_to_tr'] = df['ebitda'] / df['tr']
df['gp_to_tr'] = df['gp'] / df['tr']
df['ca_to_cl'] = df['ca'] / df['cl']
df['np_to_ta'] = df['np'] / df['ta']


df['z_score'] = 1.2*df['wc_to_ta'] + 1.4*df['re_to_ta'] + 3.3*df['ebit_to_ta'] + 0.6*df['mv_to_tl'] + 1.0*df['tr_to_ta']

df['latest_period'] = df['year'] == df.groupby('company_name')['year'].transform('max')
df['latest_period'] = df['latest_period'].astype(int)

condition = (df['latest_period'] == 1) & (df['status_label'] == 'failed')
df['default'] = np.where(condition, 1, 0)


In [9]:
features = ['wc_to_ta', 're_to_ta', 'ebit_to_ta', 'mv_to_tl', 'tr_to_ta', 'tltd_to_te', 'np', 'ebitda_to_tr', 'gp_to_tr', 'ca_to_cl', 'np_to_ta']

for column in features:
    df[column + '_pct'] = df[column].rank(method='min', pct=True)

In [10]:
df.to_csv('Data/american_bankruptcy_normalized.csv', index=False)